<div style="font-weight: bold; font-size: x-large; color: blue">***Setup***</div>

In [ ]:
import cudf
import pandas as pd

In [ ]:
#Returns missing values
def getMissing(df):
    totalRows = df.shape[0]

    temp = cudf.DataFrame({
        "Missing": df.isnull().sum(),
        "Missing %": (df.isnull().sum() / totalRows) * 100
        
    })
    
    temp = temp[temp['Missing'] > 0]
    
    return temp.sort_values(by=['Missing'], ascending=False)  

<div style="font-weight: bold; font-size: x-large; color: blue">***1. Merge Datasets***</div>

<div style="font-weight: bold; font-size: large; color: blue">1a. Housing Data:</div>

In [ ]:
#Read
housing_df = cudf.read_csv("Datasets/housing_data.csv")

#Get missing
getMissing(housing_df)

In [ ]:
#Filter rows
housing_df = housing_df[
    (housing_df['Status'] == 'Sold') & 
    (housing_df['Property Type'] == 'Single Family') &
    ((housing_df['City Name'] == 'Bakersfield') | (housing_df['City Name'] == 'Delano'))
]

#Drop columns
housing_df.drop([    
    'Status',
    'Property Type',
    #
    'Style',
    'Garage',
    'Street Number 2',    
    'RT007_ExpiredDate',
    'Unit Number',  
    #
    'Pending Date'
], axis=1, inplace=True)

#Set data types
housing_df['ML Number'] = cudf.to_numeric(housing_df['ML Number'])

#Trim
housing_df['Zipcode'] = housing_df['Zipcode'].astype(str).str.strip()

#Extract sold month and year
housing_df['Sold Date'] = cudf.to_datetime(housing_df['Sold Date'], format='%m/%d/%y')

#No need to trim
housing_df['Sold Month & Year'] = housing_df['Sold Date'].dt.year.astype(str) + "-" + housing_df['Sold Date'].dt.month.astype(str)

#Print 'Sold Month & Year'
housing_df[[
    'Sold Date',
    'Sold Month & Year'
]].head()

<div style="font-weight: bold; font-size: large; color: blue">1b. Mortgage Data:</div>

In [ ]:
#Read
mortgage_rates_df = cudf.read_csv("Datasets/mortgage_rates.csv")

#Split into month, day, year
#expand=True: Return DataFrame/MultiIndex expanding dimensionality.
temp = mortgage_rates_df['date'].str.split('/', expand=True)

#Ensure day and month have leading zeros
mortgage_rates_df['date_modified'] = temp[0].str.zfill(2) + '/' + temp[1].str.zfill(2) + '/' + temp[2]

#Extract mortgage month and year
mortgage_rates_df['date_modified'] = cudf.to_datetime(mortgage_rates_df['date_modified'], format='%m/%d/%Y')

#No need to trim
mortgage_rates_df['month_year'] = mortgage_rates_df['date_modified'].dt.year.astype(str) + "-" + mortgage_rates_df['date_modified'].dt.month.astype(str)

#Print 'month_year'
mortgage_rates_df[[
    'date',
    'date_modified',
    'month_year'
]].head()

In [ ]:
#Aggregate mortgage data
mortgage_rates_agg_df = mortgage_rates_df.groupby(['month_year']).agg(
    pmms30_mean=('pmms30', 'mean'),  
    pmms15_mean=('pmms15', 'mean')
)

#Rename
mortgage_rates_agg_df = mortgage_rates_agg_df.rename(columns={
    'pmms30_mean': 'Avg Monthly Mortgage Rate (30Y)',
    'pmms15_mean': 'Avg Monthly Mortgage Rate (15Y)'
})

#Print
mortgage_rates_agg_df.sort_values('month_year', ascending=False).head(13)

In [ ]:
#Merge
merged_df = cudf.merge(housing_df, mortgage_rates_agg_df, left_on=['Sold Month & Year'], right_on=['month_year'], how='left', suffixes=('', ''))

#Print
merged_df[[
    'Sold Month & Year',
    'Avg Monthly Mortgage Rate (30Y)',
    'Avg Monthly Mortgage Rate (15Y)'
]].sort_values('Sold Month & Year', ascending=False).head()

In [ ]:
#Drop columns
merged_df.drop([    
    'Sold Month & Year'
], axis=1, inplace=True)

<div style="font-weight: bold; font-size: large; color: blue">1c. Commute Scores:</div>

In [ ]:
#Read
commute_scores_df = cudf.read_csv("Datasets/commute_scores.csv")

#Filter columns
temp = commute_scores_df[[
    'ML Number',
    'Walk Score',
    'Transit Score',
    'Bike Score'
]]

#Set data types
temp['ML Number'] = cudf.to_numeric(temp['ML Number'])

#Print
temp.sort_values('ML Number').head()

In [ ]:
#Merge
merged_df = cudf.merge(merged_df, temp, left_on=['ML Number'], right_on=['ML Number'], how='left', suffixes=('', ''))

#Print
merged_df[[
    'ML Number',
    'Walk Score',
    'Transit Score',
    'Bike Score'
]].sort_values('ML Number').head()

<div style="font-weight: bold; font-size: large; color: blue">1d. School Ratings:</div>

In [ ]:
#Read
school_ratings_df = cudf.read_csv("Datasets/school_ratings.csv")

#Filter columns
school_ratings_subset_df = school_ratings_df[[
    'School Name',
    'Grade Classification',
    'Ratings out of 10'
]]

#Trim
school_ratings_subset_df['School Name'] = school_ratings_subset_df['School Name'].str.strip()

school_ratings_subset_df['Grade Classification'] = school_ratings_subset_df['Grade Classification'].str.strip()

In [ ]:
#Merge elementary schools
temp = school_ratings_subset_df[school_ratings_subset_df['Grade Classification'] == 'Elementary School']

temp = temp.rename(columns={'Ratings out of 10': "Elementary School Ratings"})

#Print
temp.sort_values('School Name').head()

In [ ]:
#Merge
merged_df = cudf.merge(merged_df, temp, left_on=['Elementry School'], right_on=['School Name'], how='left', suffixes=('', ''))

#Print
merged_df[[
    'Elementry School',
    'Elementary School Ratings'
]].drop_duplicates(subset=['Elementry School']).sort_values('Elementry School').head(10)

In [ ]:
#Merge junior high schools
temp = school_ratings_subset_df[school_ratings_subset_df['Grade Classification'] == 'Junior High School']

temp = temp.rename(columns={'Ratings out of 10': "Junior High School Ratings"})

#Print
temp.sort_values('School Name').head()

In [ ]:
#Merge
merged_df = cudf.merge(merged_df, temp, left_on=['Junior High School'], right_on=['School Name'], how='left', suffixes=('', ''))

#Print
merged_df[[
    'Junior High School',
    'Junior High School Ratings'
]].drop_duplicates(subset=['Junior High School']).sort_values('Junior High School').head(10)

In [ ]:
#Merge high schools
temp = school_ratings_subset_df[school_ratings_subset_df['Grade Classification'] == 'High School']

temp = temp.rename(columns={'Ratings out of 10': "High School Ratings"})

#Print
temp.sort_values('School Name').head()

In [ ]:
#Merge
merged_df = cudf.merge(merged_df, temp, left_on=['High School'], right_on=['School Name'], how='left', suffixes=('', ''))

#Print
merged_df[[
    'High School',
    'High School Ratings'
]].drop_duplicates(subset=['High School']).sort_values('High School').head(10)

In [ ]:
#Drop columns
merged_df.drop([    
    'School Name',
    'Grade Classification'
], axis=1, inplace=True)

<div style="font-weight: bold; font-size: large; color: blue">1e. Sex Offender Data:</div>

In [ ]:
#Read
sex_offender_df = cudf.read_csv("Datasets/sex_offender_data.csv")

#Trim
sex_offender_df['Zipcode'] = sex_offender_df['Zipcode'].astype(str).str.strip()

#Rename column
sex_offender_df = sex_offender_df.rename(columns={'Count': "Sex Offender Count"})

#Print
sex_offender_df.sort_values('Zipcode').head()

In [ ]:
#Merge
merged_df = cudf.merge(merged_df, sex_offender_df, left_on=['Zipcode'], right_on=['Zipcode'], how='left', suffixes=('', ''))

#Print
merged_df[[
    'Zipcode',
    'Sex Offender Count'
]].drop_duplicates(subset=['Zipcode']).sort_values('Zipcode').head()

<div style="font-weight: bold; font-size: large; color: blue">1f. Driving Distance to the Nearest Shopping Mall and Downtown:</div>

In [ ]:
#Read
distance_duration_df = cudf.read_csv("Datasets/distance_duration.csv")

#Set data types
distance_duration_df['ML Number'] = cudf.to_numeric(distance_duration_df['ML Number'])

#Drop columns
distance_duration_df.drop([
    #Google place ID
    "Shopping Mall ID",
    #It's blank
    "Shopping Mall Addr",
    "Shopping Mall Loc Types",
    #It's blank
    "Shopping Mall Loc Type",
    "Shopping Mall Latitude",
    "Shopping Mall Longitude",
    #It's false for all rows
    "Shopping Mall Permanently Closed",
    "Shopping Mall Driving Duration With Traffic in Secs",
    "Downtown Driving Duration With Traffic in Secs"
    
    
], axis=1, inplace=True)

#Rename columns
distance_duration_df = distance_duration_df.rename(columns={
    'Shopping Mall Name': 'Mall Name',
    'Shopping Mall Vicinity': 'Mall Vicinity',
    'Shopping Mall Driving Distance in Meters': 'Mall Distance (Meters)',
    'Shopping Mall Driving Duration Without Traffic in Secs': 'Mall Duration (Secs)',
    'Shopping Mall Usr Ratings Total': 'Mall Ratings Total',
    'Shopping Mall Ratings': 'Mall Ratings',    
    'Downtown Driving Distance in Meters': 'Downtown Distance (Meters)',
    'Downtown Driving Duration Without Traffic in Secs': 'Downtown Duration (Secs)'
})

#Print
distance_duration_df.sort_values('ML Number').head()

In [ ]:
#Merge
merged_df = cudf.merge(merged_df, distance_duration_df, left_on=['ML Number'], right_on=['ML Number'], how='left', suffixes=('', ''))

#Print
merged_df[[
    'ML Number',
    'Mall Name',
    'Mall Vicinity',
    'Mall Distance (Meters)',
    'Mall Duration (Secs)',
    'Mall Ratings Total',
    'Mall Ratings',
    'Downtown Distance (Meters)',
    'Downtown Duration (Secs)'
]].sort_values('ML Number').head()

<div style="font-weight: bold; font-size: x-large; color: blue">***2. Save***</div>

<div style="font-weight: bold; font-size: large; color: blue">2a. Get Missing Values:</div>

In [ ]:
#Print missing values
getMissing(merged_df)

<div style="font-weight: bold; font-size: large; color: blue">2c. Set Data Types:</div>

In [ ]:
#Print unique values
temp = [
    'DOM',
    'Bedrooms',
    'Bathrooms',
    'Year Built',
    'Walk Score',
    'Transit Score',
    'Bike Score',
    'Elementary School Ratings',
    'Junior High School Ratings',
    'High School Ratings',
    'Sex Offender Count',
    'Mall Name'
]

for c in temp:
    uniqueVals = merged_df[c].unique()

    print(c)
    print(uniqueVals)
    print('\n\n')

In [ ]:
#Numeric
merged_df['ML Number'] = merged_df['ML Number'].astype(str).str.replace(',', '').astype('int')
merged_df['Original Price'] = merged_df['Original Price'].astype(str).str.replace(',', '').astype('float')
merged_df['List Price'] = merged_df['List Price'].astype(str).str.replace(',', '').astype('float')
merged_df['Sold Price'] = merged_df['Sold Price'].astype(str).str.replace(',', '').astype('float')
merged_df['DOM'] = merged_df['DOM'].astype(str).str.replace(',', '').astype('float')
#Treat it as numeric: Although the number of bedrooms is a discrete integer, it represents a natural ordinal relationship. Treating it as a numeric variable allows the model to capture the linear or non-linear impact of additional bedrooms on home value.
merged_df['Bedrooms'] = merged_df['Bedrooms'].astype(str).str.replace(',', '').astype('float')
#Treat it as numeric: Bathrooms include fractional values (e.g., 2.5, 1.75), which suggest they should be treated as numeric. The model can learn that adding a half or a quarter bathroom affects home value in a proportional way.
merged_df['Bathrooms'] = merged_df['Bathrooms'].astype(str).str.replace(',', '').astype('float')
merged_df['Sq Foot'] = merged_df['Sq Foot'].astype(str).str.replace(',', '').astype('float')
merged_df['Lot Size'] = merged_df['Lot Size'].astype(str).str.replace(',', '').astype('float')
merged_df['Year Built'] = merged_df['Year Built'].astype(str).str.replace(',', '').astype('float')
merged_df['Latitude'] = merged_df['Latitude'].astype(str).str.replace(',', '').astype('float')
merged_df['Longitude'] = merged_df['Longitude'].astype(str).str.replace(',', '').astype('float')
merged_df['Avg Monthly Mortgage Rate (30Y)'] = merged_df['Avg Monthly Mortgage Rate (30Y)'].astype(str).str.replace(',', '').astype('float')
merged_df['Avg Monthly Mortgage Rate (15Y)'] = merged_df['Avg Monthly Mortgage Rate (15Y)'].astype(str).str.replace(',', '').astype('float')
merged_df['Walk Score'] = merged_df['Walk Score'].astype(str).str.replace(',', '').astype('float')
merged_df['Transit Score'] = merged_df['Transit Score'].astype(str).str.replace(',', '').astype('float')
merged_df['Bike Score'] = merged_df['Bike Score'].astype(str).str.replace(',', '').astype('float')
#Treat school ratings as numeric: They have a natural order (higher numbers represent better ratings), so they should be treated as numeric variables rather than categorical. Treating them as numeric allows the model to capture linear and non-linear relationships between school quality and home value.
merged_df['Elementary School Ratings'] = merged_df['Elementary School Ratings'].astype(str).str.replace(',', '').astype('float')
merged_df['Junior High School Ratings'] = merged_df['Junior High School Ratings'].astype(str).str.replace(',', '').astype('float')
merged_df['High School Ratings'] = merged_df['High School Ratings'].astype(str).str.replace(',', '').astype('float')
merged_df['Sex Offender Count'] = merged_df['Sex Offender Count'].astype(str).str.replace(',', '').astype('float')
merged_df['Mall Distance (Meters)'] = merged_df['Mall Distance (Meters)'].astype(str).str.replace(',', '').astype('float')
merged_df['Mall Duration (Secs)'] = merged_df['Mall Duration (Secs)'].astype(str).str.replace(',', '').astype('float')
merged_df['Mall Ratings Total'] = merged_df['Mall Ratings Total'].astype(str).str.replace(',', '').astype('float')
merged_df['Mall Ratings'] = merged_df['Mall Ratings'].astype(str).str.replace(',', '').astype('float')
merged_df['Downtown Distance (Meters)'] = merged_df['Downtown Distance (Meters)'].astype(str).str.replace(',', '').astype('float')
merged_df['Downtown Duration (Secs)'] = merged_df['Downtown Duration (Secs)'].astype(str).str.replace(',', '').astype('float')

#Categorical
merged_df['City Name'] = merged_df['City Name'].astype('category')
merged_df['Zipcode'] = merged_df['Zipcode'].astype('category')
merged_df['Area'] = merged_df['Area'].astype('category')
merged_df['Pool_None'] = merged_df['Pool_None'].astype('category')
merged_df['Pool_Inground'] = merged_df['Pool_Inground'].astype('category')
merged_df['Pool_Community'] = merged_df['Pool_Community'].astype('category')
merged_df['Pool_Spa'] = merged_df['Pool_Spa'].astype('category')
merged_df['Pool_Above_Ground'] = merged_df['Pool_Above_Ground'].astype('category')
merged_df['Heating_Central_AC'] = merged_df['Heating_Central_AC'].astype('category')
merged_df['Heating_Central_Heat'] = merged_df['Heating_Central_Heat'].astype('category')
merged_df['Heating_Other'] = merged_df['Heating_Other'].astype('category')
merged_df['Heating_Evaporative'] = merged_df['Heating_Evaporative'].astype('category')
merged_df['Heating_Floor/Wall_Heater'] = merged_df['Heating_Floor/Wall_Heater'].astype('category')
merged_df['Heating_Wood_Burning_Stove'] = merged_df['Heating_Wood_Burning_Stove'].astype('category')
merged_df['Heating_Gas'] = merged_df['Heating_Gas'].astype('category')
merged_df['Heating_Electric'] = merged_df['Heating_Electric'].astype('category')
merged_df['Heating_Propane'] = merged_df['Heating_Propane'].astype('category')
merged_df['Interior_Great_Room'] = merged_df['Interior_Great_Room'].astype('category')
merged_df['Interior_Formal_Dining'] = merged_df['Interior_Formal_Dining'].astype('category')
merged_df['Interior_Breakfast_Area'] = merged_df['Interior_Breakfast_Area'].astype('category')
merged_df['Interior_Indoor_Utility'] = merged_df['Interior_Indoor_Utility'].astype('category')
merged_df['Interior_Split_Wing'] = merged_df['Interior_Split_Wing'].astype('category')
merged_df['Interior_Formal_Living'] = merged_df['Interior_Formal_Living'].astype('category')
merged_df['Interior_Bonus_Room'] = merged_df['Interior_Bonus_Room'].astype('category')
merged_df['Interior_Office'] = merged_df['Interior_Office'].astype('category')
merged_df['Interior_Sep._Family_Room'] = merged_df['Interior_Sep._Family_Room'].astype('category')
merged_df['Interior_Extended_Living'] = merged_df['Interior_Extended_Living'].astype('category')
merged_df['Interior_Handicap'] = merged_df['Interior_Handicap'].astype('category')
merged_df['Interior_Basement'] = merged_df['Interior_Basement'].astype('category')
merged_df['Exterior_Brick'] = merged_df['Exterior_Brick'].astype('category')
merged_df['Exterior_Stucco'] = merged_df['Exterior_Stucco'].astype('category')
merged_df['Exterior_Wood'] = merged_df['Exterior_Wood'].astype('category')
merged_df['Exterior_Other'] = merged_df['Exterior_Other'].astype('category')
merged_df['Exterior_Steel_/_Metal'] = merged_df['Exterior_Steel_/_Metal'].astype('category')
merged_df['Exterior_Brick_Veneer'] = merged_df['Exterior_Brick_Veneer'].astype('category')
merged_df['Exterior_Wood_Frame'] = merged_df['Exterior_Wood_Frame'].astype('category')
merged_df['Other Features_Horse'] = merged_df['Other Features_Horse'].astype('category')
merged_df['Other Features_Alley'] = merged_df['Other Features_Alley'].astype('category')
merged_df['Other Features_Cul_De_Sac'] = merged_df['Other Features_Cul_De_Sac'].astype('category')
merged_df['Other Features_Corner'] = merged_df['Other Features_Corner'].astype('category')
merged_df['Other Features_Gated_Community'] = merged_df['Other Features_Gated_Community'].astype('category')
merged_df['Other Features_Adult_Community'] = merged_df['Other Features_Adult_Community'].astype('category')
merged_df['Other Features_Mountain'] = merged_df['Other Features_Mountain'].astype('category')
merged_df['Other Features_River'] = merged_df['Other Features_River'].astype('category')
merged_df['Other Features_Lake'] = merged_df['Other Features_Lake'].astype('category')
merged_df['Other Features_Golf_Course'] = merged_df['Other Features_Golf_Course'].astype('category')
merged_df['Other Features_Truck_Door'] = merged_df['Other Features_Truck_Door'].astype('category')
merged_df['Other Features_Partial_Fenced'] = merged_df['Other Features_Partial_Fenced'].astype('category')
merged_df['Other Features_Overhead_Door'] = merged_df['Other Features_Overhead_Door'].astype('category')
merged_df['Other Features_Additional_Buildings'] = merged_df['Other Features_Additional_Buildings'].astype('category')
merged_df['Other Features_Wet_Sprinklers'] = merged_df['Other Features_Wet_Sprinklers'].astype('category')
merged_df['Other Features_Security_Fence'] = merged_df['Other Features_Security_Fence'].astype('category')
merged_df['Other Features_Burglar_Alarm'] = merged_df['Other Features_Burglar_Alarm'].astype('category')
merged_df['Other Features_Smoke/Fire_Alarm'] = merged_df['Other Features_Smoke/Fire_Alarm'].astype('category')
merged_df['Other Features_Security_Lighting'] = merged_df['Other Features_Security_Lighting'].astype('category')
merged_df['Other Features_Sign'] = merged_df['Other Features_Sign'].astype('category')
merged_df['Other Features_Display_Window'] = merged_df['Other Features_Display_Window'].astype('category')
merged_df['Other Features_Laundry_Hookup'] = merged_df['Other Features_Laundry_Hookup'].astype('category')
merged_df['Other Features_Outside_Storage'] = merged_df['Other Features_Outside_Storage'].astype('category')
merged_df['Other Features_ADA_Compliant'] = merged_df['Other Features_ADA_Compliant'].astype('category')

#Date & time
merged_df['List Date'] =  cudf.to_datetime(merged_df['List Date'], format='%m/%d/%y')
merged_df['Sold Date'] = cudf.to_datetime(merged_df['Sold Date'], format='%Y-%m-%d')

#String
merged_df['Street Number 1'] = merged_df['Street Number 1'].astype('str')
merged_df['Street Name'] = merged_df['Street Name'].astype('str')
merged_df['Cross Street'] = merged_df['Cross Street'].astype('str')

merged_df['Elementry School'] = merged_df['Elementry School'].astype('str')
merged_df['Junior High School'] = merged_df['Junior High School'].astype('str')
merged_df['High School'] = merged_df['High School'].astype('str')

merged_df['Mall Name'] = merged_df['Mall Name'].astype('str')
merged_df['Mall Vicinity'] = merged_df['Mall Vicinity'].astype('str')

#Print data types
merged_df.info()

In [ ]:
#Print values
pd.set_option('display.max_columns', None)

merged_df.head(10)

<div style="font-weight: bold; font-size: large; color: blue">2d. Save:</div>

In [ ]:
#Save data frame
merged_df.to_feather("Datasets/Merged.feather")

In [ ]:
#Save sample
merged_df.head(200).to_csv("Temp/Merged_Sample.csv")

In [ ]:
#Test
cudf.read_feather("Datasets/Merged.feather").info()